In [1]:
from tokenize import group

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy.dialects.mssql.information_schema import columns
import hashlib

plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine, column
import json
import datetime
from datetime import timedelta ,time, datetime
import re
import os
import sys
import gc
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Data_Clean
pd.set_option('display.max_columns', None)


# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
clean = Data_Clean()

# 查询数据

In [2]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money, tod.product_id
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
, om.order_type, om.union_rent_tag,tprm.rent_received 已收租金 
,replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.rejected_ato_risk') end,'"','') as rejected_ato_risk
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id



left join db_credit.risk risk on risk.order_id=om.id


where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')

# and om.create_time >= CURDATE()
and om.create_time >= DATE_ADD(CURDATE(), INTERVAL -7 DAY)
and om.create_time < DATE_ADD(CURDATE(), INTERVAL 1 DAY)
and hour(om.create_time) >= 10
and hour(om.create_time) < 17
'''

# df_order = clean.query(sql1)
# df_order.shape

In [3]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected

from db_credit.risk
'''
# df_risk = clean.query(sql_risk)

In [4]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
# df_risk_examine = clean.query(sql3)
# df_re = df_risk_examine.copy()
# df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
# df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

In [5]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
# df_ra = clean.query(sql_ra)
# df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
# df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

In [6]:
def process_live_conversion_data():
    # 执行sql语句，获取数据
    df_order = clean.query(sql1)
    df_risk = clean.query(sql_risk)
    df_risk_examine = clean.query(sql3)
    df_ra = clean.query(sql_ra)
    
    df_re = df_risk_examine.copy()
    df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
    df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
    
    df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
    df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)
    # 数据清洗和处理
    df = df_order.copy()
    df = df[df.type!=4]
    
    # 下单日期
    df["下单日期"]=df["create_time"].dt.date
    df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
    df["月份"]=df["下单日期"].dt.month
    df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
    df['年份'] = df["下单日期"].dt.year
    df['hour'] = df['create_time'].dt.hour
    df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
    
    # 处理备注信息
    df = df[df['sku_attributes'].notnull()]
    df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
    df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
    df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
    df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
    df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
    df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
    df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
    df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")


    df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
    df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
    # 物流状态=5，退货拒签

    # 去重、剔除商家
    df = clean.order_drop_duplicates(df)
    df = clean.drop_merchant(df)
    
    # 定义状态
    # note 2025-02-24添加
    df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
    # , 'union_rent_tag', 'union_rent_rejected'
    # 判断 前置拦截   机审强拒   出库前风控强拒
    df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                            |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)

    # note 2025-02-24修改
    df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
    df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

    df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)
    df = clean.status_node(df)

    # 剔除拒量
    df2 = df.copy()
    df2 = clean.drop_rejected_merchant(df2)
    df2 = df2[df2.是否拒量==0]
    day_name_cn_mapping = {
        'Monday': '星期一',
        'Tuesday': '星期二',
        'Wednesday': '星期三',
        'Thursday': '星期四',
        'Friday': '星期五',
        'Saturday': '星期六',
        'Sunday': '星期日'
    }
    # 获取转化数据
    df_zfb = df[df['activity_name'].str.contains('支付宝直播商品')==True]
    df_zfb2 = df2[df2['activity_name'].str.contains('支付宝直播商品')==True]
    df_zfb_group = all_models.data_group(df_zfb, df_zfb2, df_risk_examine, '下单日期')

    df_zfb_group['出库（按下单时间）'] = df_zfb_group['出库']
    df_zfb_group['出库（按转化时间）'] = df_zfb_group['出库']
    df_zfb_group = df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）', '出库（按转化时间）','进件出库率','订单出库率']]
    df_zfb_group1 = df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','订单出库率']].reset_index(names=['下单日期'])
    
    return df_zfb_group1
    

# 执行定时任务

In [7]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler

hour = 17
minute = 45
def my_job1():
    # 获取转化数据
    df_zfb_group1 = process_live_conversion_data()
    path = r'\\digua\迪瓜租机\002数据监测\3.转化数据\支付宝直播截止到五点数据/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    print(f'执行定时任务：现在是{Today1}的45分')

    with pd.ExcelWriter(path + f'支付宝直播截止至17点转化数据{Today1}.xlsx', engine='openpyxl') as writer:
        df_zfb_group1.to_excel(writer, sheet_name='支付宝直播', index=False)

    del df_zfb_group1
    gc.collect()
    print("回收内存执行完毕！\n")
    
# my_job1()
scheduler1 = BackgroundScheduler()
# 添加一个cron任务，每天的10:30执行


scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# 启动scheduler1
scheduler1.start()


执行定时任务：现在是2025092217的45分
回收内存执行完毕！

执行定时任务：现在是2025092317的45分
回收内存执行完毕！

执行定时任务：现在是2025092417的45分
回收内存执行完毕！

执行定时任务：现在是2025092517的45分
回收内存执行完毕！

